In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [4]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi")

<Experiment: artifact_location='./mlruns/1', creation_time=1715985426953, experiment_id='1', last_update_time=1715985426953, lifecycle_stage='active', name='nyc-taxi', tags={}>

In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_dataframe('data/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('data/yellow_tripdata_2022-02.parquet')

In [15]:
df_train = df_train.iloc[:5000]

In [16]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [18]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

1500.1035079271394

In [20]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

<1x20667 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [31]:
with mlflow.start_run():
    mlflow.set_tag("developer","jdilger")
    mlflow.log_param("train-data-path","/a/b/c/c")
    mlflow.log_param("test-data-path","/a/b/c/c")

    alpha = 0.001
    mlflow.log_param("alpha",alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse",rmse)
    mlflow.log_artifact(local_path="models/lin_reg.bin", 
                        artifact_path="models_pickle/lin_reg.bin")

In [24]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


In [25]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [26]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [28]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), #[0.05 - 1]
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/johndilger/mambaforge/envs/geospatial/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [18:23:41] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1713397655897/work/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.34758                           
[1]	validation-rmse:7.55322                           
[2]	validation-rmse:7.02169                           
[3]	validation-rmse:6.68755                           
[4]	validation-rmse:6.45961                           
[5]	validation-rmse:6.34744                           
  0%|          | 0/50 [00:01<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [29]:
params = {
 'learning_rate':	0.05063907284626492,
'max_depth':	5,
'min_child_weight':	8.369692235045367,
'objective':	'reg:linear',
'reg_alpha':	0.287105827816395,
'reg_lambda':	0.017805715312008284,
'seed':	42   ,
}
mlflow.xgboost.autolog()
booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)
y_pred = booster.predict(valid)
rmse = mean_squared_error(y_val, y_pred, squared=False)

2024/05/17 18:26:42 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.
2024/05/17 18:26:42 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '27f95db42b9246c9b041b71a1edd507a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:9.37869


/Users/johndilger/mambaforge/envs/geospatial/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [18:26:42] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1713397655897/work/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:9.11850
[2]	validation-rmse:8.87586
[3]	validation-rmse:8.64981
[4]	validation-rmse:8.43935
[5]	validation-rmse:8.24353
[6]	validation-rmse:8.06136
[7]	validation-rmse:7.89248
[8]	validation-rmse:7.73556
[9]	validation-rmse:7.59022
[10]	validation-rmse:7.45541
[11]	validation-rmse:7.33046
[12]	validation-rmse:7.21479
[13]	validation-rmse:7.10786
[14]	validation-rmse:7.00863
[15]	validation-rmse:6.91690
[16]	validation-rmse:6.83228
[17]	validation-rmse:6.75451
[18]	validation-rmse:6.68312
[19]	validation-rmse:6.61668
[20]	validation-rmse:6.55601
[21]	validation-rmse:6.49920
[22]	validation-rmse:6.44701
[23]	validation-rmse:6.39908
[24]	validation-rmse:6.35468
[25]	validation-rmse:6.31422
[26]	validation-rmse:6.27571
[27]	validation-rmse:6.24060
[28]	validation-rmse:6.20780
[29]	validation-rmse:6.17805
[30]	validation-rmse:6.14977
[31]	validation-rmse:6.12341
[32]	validation-rmse:6.09929
[33]	validation-rmse:6.07729
[34]	validation-rmse:6.05633
[35]	validation-rmse:6.

2024/05/17 18:27:28 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/17 18:27:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/johndilger/mambaforge/envs/geospatial/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [18:27:28] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1713397655897/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
2024/05/17 18:27:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/johndilger/mambaforge/envs/geospatial/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [ ]:
params = {
 'learning_rate':	0.05063907284626492,
'max_depth':	5,
'min_child_weight':	8.369692235045367,
'objective':	'reg:linear',
'reg_alpha':	0.287105827816395,
'reg_lambda':	0.017805715312008284,
'seed':	42   ,
}
mlflow.xgboost.autolog()
booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)
y_pred = booster.predict(valid)
rmse = mean_squared_error(y_val, y_pred, squared=False)

2024/05/17 18:26:42 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.
2024/05/17 18:26:42 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '27f95db42b9246c9b041b71a1edd507a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:9.37869


/Users/johndilger/mambaforge/envs/geospatial/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [18:26:42] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1713397655897/work/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:9.11850
[2]	validation-rmse:8.87586
[3]	validation-rmse:8.64981
[4]	validation-rmse:8.43935
[5]	validation-rmse:8.24353
[6]	validation-rmse:8.06136
[7]	validation-rmse:7.89248
[8]	validation-rmse:7.73556
[9]	validation-rmse:7.59022
[10]	validation-rmse:7.45541
[11]	validation-rmse:7.33046
[12]	validation-rmse:7.21479
[13]	validation-rmse:7.10786
[14]	validation-rmse:7.00863
[15]	validation-rmse:6.91690
[16]	validation-rmse:6.83228
[17]	validation-rmse:6.75451
[18]	validation-rmse:6.68312
[19]	validation-rmse:6.61668
[20]	validation-rmse:6.55601
[21]	validation-rmse:6.49920
[22]	validation-rmse:6.44701
[23]	validation-rmse:6.39908
[24]	validation-rmse:6.35468
[25]	validation-rmse:6.31422
[26]	validation-rmse:6.27571
[27]	validation-rmse:6.24060
[28]	validation-rmse:6.20780
[29]	validation-rmse:6.17805
[30]	validation-rmse:6.14977
[31]	validation-rmse:6.12341
[32]	validation-rmse:6.09929
[33]	validation-rmse:6.07729
[34]	validation-rmse:6.05633
[35]	validation-rmse:6.

2024/05/17 18:27:28 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/17 18:27:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/johndilger/mambaforge/envs/geospatial/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [18:27:28] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1713397655897/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
2024/05/17 18:27:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/johndilger/mambaforge/envs/geospatial/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [32]:
with mlflow.start_run():
    params = {
    'learning_rate':	0.05063907284626492,
    'max_depth':	5,
    'min_child_weight':	8.369692235045367,
    'objective':	'reg:linear',
    'reg_alpha':	0.287105827816395,
    'reg_lambda':	0.017805715312008284,
    'seed':	42   ,
    }
    mlflow.log_params(params=params)
    
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=10,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:9.37869


/Users/johndilger/mambaforge/envs/geospatial/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [19:28:49] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1713397655897/work/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:9.11850
[2]	validation-rmse:8.87586
[3]	validation-rmse:8.64981
[4]	validation-rmse:8.43935
[5]	validation-rmse:8.24353
[6]	validation-rmse:8.06136
[7]	validation-rmse:7.89248
[8]	validation-rmse:7.73556
[9]	validation-rmse:7.59022


2024/05/17 19:28:50 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/17 19:28:50 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/johndilger/mambaforge/envs/geospatial/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [19:28:50] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1713397655897/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
/Users/johndilger/mambaforge/envs/geospatial/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [19:28:52] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1713397655897/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will def